In [1]:
from dask.diagnostics import ProgressBar

In [2]:
import dask.dataframe as dd

In [3]:
import numpy as np
import pandas as pd

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from dask import delayed

In [6]:
df_demography = dd.read_csv('Telco_customer_churn_demographics.csv')
df_location = dd.read_csv('Telco_customer_churn_location.csv')
df_service = dd.read_csv('Telco_customer_churn_services updated.csv')
df_status = dd.read_csv('Telco_customer_churn_status.csv')

In [7]:
type(df_status)

dask.dataframe.core.DataFrame

In [8]:
# df_status.describe().compute()

In [9]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.75 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56579,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 15.75 GiB
Comm: tcp://127.0.0.1:56607,Total threads: 2
Dashboard: http://127.0.0.1:56611/status,Memory: 3.94 GiB
Nanny: tcp://127.0.0.1:56583,


In [10]:
df_location = df_location.drop(['Lat Long','Country'], axis=1)

In [11]:
# REfer to status notebook

df_status = df_status.drop(['Satisfaction Score','Customer Status','Churn Category','Churn Reason','Churn Score', 'Churn Label' ], axis = 1)

In [12]:
result = dd.concat([df_demography, df_location, df_status,df_service ], axis=1)

C:\Users\inzamam.safi\Anaconda3\lib\site-packages\dask\dataframe\multi.py:1222: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


In [13]:
# result.head()

In [14]:
# dropping duplicate column
result = result.loc[:,~result.columns.duplicated()]

In [15]:
# dropping nan values
df= result.dropna()

In [16]:
# checking for the duplicates values.
df = df.drop_duplicates()

In [17]:
# df.head()

In [18]:
df=df.drop(['Count'], axis = 1)

In [19]:
# correlation = df.corr()
# plt.figure(figsize=(15,16))
# sns.heatmap(correlation, cbar=True, square=True, fmt = '.2f', annot = True, annot_kws={'size':15}, cmap = 'coolwarm')

In [20]:
# Columns to drop
df=df.drop(['Longitude','Latitude','Tenure in Months','Total Charges','Total Services','Tenure in Months','Total Long Distance Charges'], axis = 1)

In [21]:
df=df.drop(['State'],axis=1)

In [22]:
# df.shape[0].compute()

In [23]:
type(df)

dask.dataframe.core.DataFrame

In [24]:
# df.head()

In [25]:
df=df.drop(['City'],axis=1)

In [26]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 35 entries, Customer ID to CLTV
dtypes: object(23), float64(4), int64(8)

In [27]:
df=df.drop(['Customer ID', 'Quarter'],axis=1)

In [28]:
# df.head()

### Outlier handling

In [29]:
def IQR(x):
    Q1 = x.quantile(0.25)
    Q3 = x.quantile(0.75)
    IQR = Q3 - Q1
    low_lim = Q1 - 1.5 * IQR
    up_lim = Q3 + 1.5 * IQR
    df1= df[x <= up_lim]
    return df1

dfx1=delayed(IQR)(df['Total Revenue'])

dfx2=delayed(IQR)(df['Avg Monthly GB Download'])

def merging(dfx1,dfx2):
    return dd.merge(dfx1,dfx2, how='inner')

df_combo=delayed(merging)(dfx1,dfx2)

df=df_combo.compute()

df.head()

,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,Zip Code,Churn Value,Referred a Friend,...,Streaming Music,Unlimited Data Pack,Contract,Paperless Billing,Payment Method,Average Monthly Charge,Total Refunds,Total Extra Data Charges,Total Revenue,CLTV
0,Male,78,No,Yes,No,No,0,90022,1,No,...,No,No,Month-to-Month,Yes,Bank Withdrawal,39.65,0.00,20,59.65,5433
1,Female,74,No,Yes,Yes,Yes,1,90063,1,Yes,...,No,Yes,Month-to-Month,Yes,Credit Card,80.65,0.00,0,1024.10,5302
2,Male,98,No,Yes,No,Yes,3,90065,1,No,...,Yes,Yes,Month-to-Month,Yes,Bank Withdrawal,95.45,45.61,0,1910.88,3179
3,Female,78,No,Yes,Yes,Yes,1,90303,1,Yes,...,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.50,13.43,0,2995.07,5337
4,Female,80,No,Yes,Yes,Yes,1,90602,1,Yes,...,No,Yes,Month-to-Month,Yes,Bank Withdrawal,76.50,0.00,0,3102.36,2793


### Handling Categorical data

In [30]:
# df_cat = pd.Categorical(df)

In [31]:
df_dum = dd.get_dummies(df.categorize())
# df_dum.head()

In [32]:
### MODEL 1

In [33]:
# seprating predictor and target variable
y = df_dum['Churn Value'].values
X = df_dum.drop(columns = ['Churn Value'])

In [34]:
type(X)

dask.dataframe.core.DataFrame

In [35]:
type(y)

dask.array.core.Array

In [36]:
y = dd.io.from_dask_array(y)

In [37]:
type(y)

dask.dataframe.core.Series

In [38]:
# Normalizing the data to avoid bias with features having high values.
X=(X-X.min())/(X.max()-X.min())


In [39]:
# from dask_ml.linear_model import LogisticRegression
from dask_ml.model_selection import train_test_split as train_test_split_dask

In [40]:
# from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [41]:
import joblib
model = LogisticRegression(max_iter=1000)
X_train, X_test, y_train, y_test = train_test_split_dask(X, y, test_size=0.2, random_state=40)
with joblib.parallel_backend("dask"):
    result = model.fit(X_train, y_train)

C:\Users\inzamam.safi\Anaconda3\lib\site-packages\dask_ml\model_selection\_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [42]:
from sklearn import metrics
prediction_test = model.predict(X_test)
# Print the prediction accuracy
print (metrics.accuracy_score(y_test, prediction_test))

print("------")
print("------")

from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test,prediction_test)) 

print("------")
print("------")

## Classification_report for Logistic
print('TRAIN: Classification_report for Logistic \n')
print(classification_report(y_train, model.predict(X_train)))
print('TEST: Classification_report for Logistic \n')
print(classification_report(y_test, prediction_test))

0.8464536243180046
------
------
[[877  83]
 [114 209]]
------
------
TRAIN: Classification_report for Logistic 

              precision    recall  f1-score   support

           0       0.88      0.91      0.89      3915
           1       0.72      0.66      0.69      1425

    accuracy                           0.84      5340
   macro avg       0.80      0.78      0.79      5340
weighted avg       0.84      0.84      0.84      5340

TEST: Classification_report for Logistic 

              precision    recall  f1-score   support

           0       0.88      0.91      0.90       960
           1       0.72      0.65      0.68       323

    accuracy                           0.85      1283
   macro avg       0.80      0.78      0.79      1283
weighted avg       0.84      0.85      0.84      1283



In [ ]:
# MODEL 2

In [43]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [44]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from datetime import datetime

In [45]:
from dask_ml.model_selection import HyperbandSearchCV

In [46]:
params = {'n_estimators': [200,300,400,500,600,700],
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
          'eta':[0.01,0.02,0.3],
          'lambda':[0.8,1,1.2,1.4],
          'colsample_bynode':[0.6, 0.8, 1.0],
          'colsample_bylevel':[0.6, 0.8, 1.0]
        }

In [47]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [48]:
from dask_ml.wrappers import Incremental

In [49]:
xgb  = Incremental(XGBClassifier())

In [55]:
folds = 5
param_comb = 10

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = HyperbandSearchCV(xgb, parameters=params, max_iter=param_comb, scoring='roc_auc', test_size=0.1 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
with joblib.parallel_backend("dask"):
    random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

ValueError: Invalid parameter 'subsample' for estimator Incremental(estimator=XGBClassifier(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone

In [ ]:
print('\n Best hyperparameters:')
print(random_search.best_params_)

In [ ]:
# Testing the parameter

In [ ]:
model = XGBClassifier(**random_search.best_params_)

In [ ]:
with joblib.parallel_backend("dask"):
    result = model.fit(X_train, y_train)

In [ ]:
preds_xgb = model.predict(X_test)
print('accuracy',accuracy_score(y_test, preds_xgb))

print("------")
print("------")

## Classification_report for Logistic
print('Classification_report for xgboost \n')
print(classification_report(y_train, model.predict(X_train)))
print('Classification_report for xgboost \n')
print(classification_report(y_test, preds_xgb))

In [ ]:
# with joblib.parallel_backend("dask"):
#     result = model.fit(X_train, y_train)

In [ ]:
# preds_xgb = model.predict(X_test)
# print('accuracy',accuracy_score(y_test, preds_xgb))

# print("------")
# print("------")

# ## Classification_report for Logistic
# print('Classification_report for xgboost \n')
# print(classification_report(y_train, model.predict(X_train)))
# print('Classification_report for xgboost \n')
# print(classification_report(y_test, preds_xgb))

In [ ]:
# PROBLEM Ocurring:
# 1. stratified problem while splitting

In [ ]:
# Problem that I faced but got resolved:
# 1. model in parallel was thowing earlier
# 2. Using dealyed for multiple function then again using dealyed for combine operation.
# 3. 